# Validación e importación de los metadatos de elementos<a class="anchor" id="top"></a>

En este cuaderno, retomará lo visto en `01_Validating_and_Importing_User_Item_Interaction_Data.ipynb` para crear un conjunto de datos de los metadatos de elementos operativo. Esto permitirá trabajar con filtros, así como también, admitir los algoritmos `User Personalization` o `HRNN-Metadata` más adelante.


Para ejecutar este cuaderno, es necesario haber ejecutado el cuaderno anterior, `01_Validating_and_Importing_User_Item_Interaction_Data`, en el que creó un conjunto de datos e importó datos de interacción a Amazon Personalize. Al final de ese cuaderno, guardó algunos de los valores de las variables, que ahora debe cargar en este cuaderno.

In [1]:
%store -r

## Prepare sus metadatos de elementos <a class="anchor" id="prepare"></a>
[Regresar al principio](#top)

Lo que debe hacer a continuación es cargar los datos y confirmar que los datos se encuentran en buen estado. Luego, debe guardarlos en un archivo CSV donde estén listos para su uso con Amazon Personalize.

Para comenzar, importe una recopilación de bibliotecas de Python que se utilicen normalmente en la ciencia de datos.

In [2]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

Luego, abra el archivo de datos y observe varias de las primeras filas.

In [3]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
original_data.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


Esto no nos explica mucho acerca del conjunto de datos, por lo que buscaremos más información sin procesar. Podemos observar que, en general, los géneros se agrupan entre sí, y eso está bien, porque Personalize admite esta estructura.

In [5]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


De esta observación, podemos decir que hay un total de (62 000+ para las 9 742 completas para las pequeñas) entradas en el conjunto de datos, con 3 columnas.

Este conjunto de datos es muy pequeño, solo posee el ID de la película, el título y la lista de géneros aplicables a cada entrada. Sin embargo, hay datos adicionales disponibles en el conjunto de datos de MovieLens. Por ejemplo, el título incluye el año de lanzamiento de las películas. Hagamos que eso sea una columna nueva de metadatos.

In [6]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


Desde la perspectiva de los metadatos de elementos, solo queremos incluir información relevante para el modelo de prueba o resultados de filtros, omitiremos el título, pero conservaremos la información del género.

In [7]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

,movieId,genres,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Adventure|Children|Fantasy,1995
2,3,Comedy|Romance,1995
3,4,Comedy|Drama|Romance,1995
4,5,Comedy,1995


Luego de manipular los datos, siempre confirme si el formato de datos ha sido modificado.

In [8]:
itemmetadata_df.dtypes

movieId     int64
genres     object
year       object
dtype: object

Amazon Personalize tiene una columna predeterminada para `ITEM_ID` que lo guiará a nuestro `movieId`. Ahora, podemos complementarlo con más información al especificar `GENRE` también.

In [9]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

¡Eso es todo! A esta altura, los datos están listos y solo necesitamos guardarlos como un archivo CSV.

In [10]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

In [11]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Crear el conjunto de datos

Primero, defina un esquema para indicar a Amazon Personalize el tipo de conjunto de datos que está cargando. Existen varias palabras clave reservadas y obligatorias, necesarias para el esquema, según el tipo de conjunto de datos. Para obtener más información, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Aquí, creará un esquema para los datos de metadatos de elementos, para lo que se necesitan los campos `ITEM_ID` y `GENRE`. En el esquema, estos deben estar definidos en el mismo orden que aparecen el conjunto de datos.

In [12]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },{
            "name": "YEAR",
            "type": "int",
        },
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:136455442858:schema/personalize-poc-movielens-item",
  "ResponseMetadata": {
    "RequestId": "221f8e73-aa81-4650-96c0-20e3e5bda29d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 22:29:29 GMT",
      "x-amzn-requestid": "221f8e73-aa81-4650-96c0-20e3e5bda29d",
      "content-length": "96",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Con un esquema creado, puede crear un conjunto de datos dentro del grupo de conjuntos de datos. Nota: Esto aún no carga los datos. Esto sucederá unos pasos más adelante.

In [13]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:136455442858:dataset/personalize-poc-movielens/ITEMS",
  "ResponseMetadata": {
    "RequestId": "c86476eb-21e6-4743-9f26-228be0ff5e03",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 22:29:31 GMT",
      "x-amzn-requestid": "c86476eb-21e6-4743-9f26-228be0ff5e03",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Cargar los datos a S3

Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos. 

In [14]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

## Importe los metadatos de elementos <a class="anchor" id="import"></a>
[Regresar al principio](#top)

Anteriormente, creó el grupo de conjuntos de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos del bucket de S3 en el conjunto de datos de Amazon Personalize. 

In [15]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:136455442858:dataset-import-job/personalize-poc-item-import1",
  "ResponseMetadata": {
    "RequestId": "ca29c3d7-ecd9-441d-a70e-72956182e6ab",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 22:29:37 GMT",
      "x-amzn-requestid": "ca29c3d7-ecd9-441d-a70e-72956182e6ab",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de que podamos utilizar el grupo de conjuntos de datos, el trabajo de importación debe estar activo. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del trabajo de importación a cada segundo, por hasta un máximo de 6 horas.

La importación de los datos puede llevar algo de tiempo, según el tamaño del conjunto de datos. En este taller, el trabajo de importación debería tomar alrededor de 15 minutos.

In [16]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 98.8 ms, sys: 2.05 ms, total: 101 ms
Wall time: 15min 1s


Una vez completa la importación, puede activar los filtros para sus recomendaciones, así como también admitir `HRNN-Metadata`. Ejecute la celda a continuación antes de pasar a almacenar algunos valores para su uso en los siguientes cuadernos. Después de completar esa celda, abra el cuaderno `03_Creating_and_Evaluating_Solutions.ipynb` para continuar.

In [17]:
%store items_dataset_arn
%store itemmetadataschema_arn

Stored 'items_dataset_arn' (str)
Stored 'itemmetadataschema_arn' (str)
